# 데이터 준비

In [3]:
import pandas as pd
import numpy as np

In [4]:
import os
os.listdir('./2015_to_2017')

['NewsResult_20150101-20151231.csv',
 'NewsResult_20160101-20161231.csv',
 'NewsResult_20170101-20171231.csv']

In [5]:
df = pd.read_csv('./2015_to_2017/NewsResult_20150101-20151231.csv')
new_df = pd.read_csv('./2015_to_2017/NewsResult_20160101-20161231.csv')
df = pd.concat([df, new_df], axis=0)
new_df = pd.read_csv('./2015_to_2017/NewsResult_20170101-20171231.csv')
df = pd.concat([df, new_df], axis=0)

In [ ]:
df = pd.read_csv('./2015_to_2017/NewsResult_20150101-20151231.csv')

for path in os.listdir('./2015_to_2017')[1:]:
    new_df = pd.read_csv('./2015_to_2017/' + path)
    df = pd.concat([df, new_df], axis=0)
df.shape

In [ ]:
환율 = pd.read_csv('어쩌면최종데이터.csv')

# ✅ 결측값 개수 및 비율 계산
null_df = pd.DataFrame({
    '결측값 개수': 환율.isnull().sum(),
    '결측 비율 (%)': 환율.isnull().mean() * 100
})

# ✅ 'alpha'가 컬럼명에 포함된 것 중에서만 결측 비율 5% 이상인 컬럼 필터링
alpha_cols = [col for col in 환율.columns if 'alpha' in col]
alpha_null_df = null_df.loc[alpha_cols]
drop_cols = alpha_null_df[alpha_null_df['결측 비율 (%)'] >= 5].index.tolist()

# ✅ 해당 alpha 컬럼들만 삭제
환율 = 환율.drop(columns=drop_cols)

# ✅ 남은 결측값 전체 삭제 (단, alpha 외 컬럼 포함)
환율 = 환율.dropna()

환율 = 환율.sort_values(by='Date').reset_index(drop=True)
print('완료')

In [6]:
print('수집한 데이터는 약', int(df.shape[0]/10000), '만 건입니다')

수집한 데이터는 약 434 만 건입니다


In [4]:
df.columns

Index(['Unnamed: 0', '뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2',
       '통합 분류3', '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관',
       '키워드', '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부', 'file_date_range'],
      dtype='object')

# EPU 직접 수집한 데이터로 계산
## 처리 순서
- 언론사 n개 정의
- 경제, 불확실성, 정책 카테고리에 각각 하나의 용어를 공동으로 포함하는 기사를 수집
- 기간에 맞게 뉴스 기사에 대한 데이터를 수집
- 신문사, 월별로 관련 기사 수를 집계 후, 동일 신문의 총 기사 수로 집계하여 상대 빈도 수를 얻음
- 각 신문의 상대 EPU 수를 전체 표본 기간의 단위 표준 편차로 표준화
- 신문의 표준화 시리즈를 월별로 평균하여 전체 월간 한국 EPU지수
- EPU 디수를 표본 기간 중 평균 100으로 정규화

# 월별, 논문에 있는 EPU만 했을 때
- 직접개발 0.361912	< barker 0.412254

In [8]:
# 경제
e = ['경제','경기']
# 정책
p = ['불확실','불확실성','리스크','불투명','불안','우려','걱정']
# 불확실
u = '정부 청와대 국무회의 국회 의회 당국 한국은행 한은 중앙은행 기획재정부 기재부 금융위원회 금융위 정책 재정 입법 법안 법률 예산 재정 세금 규제 규정 적자 부족 부채 채무 연방준비제도 연준 Fed FRB 구조개혁 구조조정'.split()
금융정책_불확실성_지수 =['통화정책','한국은행','한은','금융완화','추가완화','양적완화','질적완화','통화긴축','유동성긴축','마이너스 금리','통화할인율','재할인율','통화 운용','공개시장조작','물가안정목표','물가목표','금융통화위원회','금통위','금융통화운영위원회','금융통의','기준금리','정책금리','금리정책','통화당국','환금리']
재정정책_불확실성_지수 = [
    '재정정책', '정부 예산', '추가경정예산', '추경', '일반 회계', '특별 회계', '재정적자', '재정수지',
    '세금', '조세', '정부지출', '재정지출', '사회보장성 지출', '국민연금 보험료', '국민건강 보험료',
    '의료비 지출', '간병비 지출', '의료보수수가', '의료수가', '공무원급여', '공적개발원조', 'ODA',
    '국방비', '군비', '국채 발행 잔여', '공공부문 부채', '재정부채', '국채', '정부부채', '지방채',
    '경기부양', '경기부양책'
]
무역정책_불확실성_지수 = [
    '통상문제', '무역문제', '비관세장벽', '수입제한', '포괄통상법', '종합무역법',
    '무역정책', '통상정책', '무역협상', '세계무역기구', 'WTO',
    '관세 및 무역에 관한 일반협정', 'GATT', '가트', '관세 인하',
    '무역자유화', '수입자유화', '시장접근', '무역협정', '통상협정',
    '환태평양경제동반자협정', 'TPP', '경제동반자협정', '경제파트너협정',
    '경제 파트너십 협정', 'EPA', '자유무역협정', 'FTA', '무역분쟁',
    '관세', '우루과이라운드', '도하라운드', '덤핑'
]
환율정책_불확실성_지수 = [
    '외환정책', '환율정책', '시장개입', '외환시장 개입', '외환당국 개입',
    '미세조정', '환율 안정책', '환율 안정 조치', '환율 변동성 완화',
    '환율 조작', '외환보유고', '외환보유액', '달러매도', '원화 매입 개입'
]

In [9]:
import re

cols = ['키워드', '특성추출(가중치순 상위 50개)', '본문', '제목']          # 검색 대상 열
joined = df[cols].fillna('').agg(' '.join, axis=1)                          # 네 열을 하나로 합침

cond1 = joined.str.contains('|'.join(map(re.escape, e)))                    # 경제·경기
cond2 = joined.str.contains('|'.join(map(re.escape, p)))                    # 불확실 관련
cond3 = joined.str.contains('|'.join(map(re.escape, u)))                    # 정책(정부 등)
                 # 정책(정부 등)

# 세 조건을 모두 만족하는 기사만 추리기
df_epu = df[cond1&cond2&cond3]

In [10]:
print(df_epu.shape, '데이터를 사용하게 되었습니다')
print('전체의', df_epu.shape[0]/df.shape[0] *100, '%입니다.')

(86529, 21) 데이터를 사용하게 되었습니다
전체의 1.993013661272669 %입니다.


In [11]:
df_epu.columns

Index(['Unnamed: 0', '뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2',
       '통합 분류3', '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관',
       '키워드', '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부', 'file_date_range'],
      dtype='object')

In [12]:
# 신문사 월별로 집계
df_epu['일자'] = pd.to_datetime(df_epu['일자'], format='%Y%m%d')
df_epu['일자']= df_epu['일자'].dt.strftime('%Y-%m-01')
집계 = pd.DataFrame(df_epu.groupby(['일자','언론사']).size().reset_index())
집계.columns = ['일자','언론사','epu기사수']

C:\Users\samsung\AppData\Local\Temp\ipykernel_26620\353623932.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epu['일자'] = pd.to_datetime(df_epu['일자'], format='%Y%m%d')
C:\Users\samsung\AppData\Local\Temp\ipykernel_26620\353623932.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epu['일자']= df_epu['일자'].dt.strftime('%Y-%m-01')


In [13]:
# 각 신문의 상대 기사 수
전체집계 = pd.DataFrame(df.groupby(['일자','언론사']).size().reset_index())
전체집계['일자'] = pd.to_datetime(전체집계['일자'], format='%Y%m%d')
전체집계['일자'] = 전체집계['일자'].dt.strftime('%Y-%m-01')
전체집계.columns =  ['일자','언론사','전체기사수']
전체집계 = 전체집계.groupby(['일자','언론사'])['전체기사수'].sum().reset_index()
전체집계= 전체집계.merge(집계, on=['일자','언론사'])
전체집계['상대빈도수'] = 전체집계['epu기사수']/전체집계['전체기사수']

In [14]:
# 각 신문의 상대 EPU 수를 전체 표본 기간의 단위 표준 편차로 표준화
전체집계['표준화빈도'] = 전체집계['상대빈도수'] / 전체집계.groupby('언론사')['상대빈도수'].transform('std')

In [15]:
# 신문의 표준화 시리즈를 월별로 평균하여 전체 월간 한국 EPU지수
전체집계['일자'] = pd.to_datetime(전체집계['일자'], format='%Y-%m-01')

# ② 신문 간 단순 평균 → 월별 EPU
monthly_epu = (전체집계
               .groupby('일자')['표준화빈도']
               .mean())

In [16]:
# EPU 지수를 표본 기간 중 평균 100으로 정규화
korea_epu = (monthly_epu / monthly_epu.mean() * 100).rename('Korea_EPU')

In [17]:
korea_epu

일자
2015-01-01    102.836415
2015-02-01     83.524611
2015-03-01     90.767212
2015-04-01     78.264460
2015-05-01     80.920413
2015-06-01    138.176954
2015-07-01     93.882377
2015-08-01    109.402812
2015-09-01     94.984700
2015-10-01     82.695156
2015-11-01     86.096626
2015-12-01    116.157674
2016-01-01    143.724545
2016-02-01    129.310440
2016-03-01     90.072357
2016-04-01     99.627492
2016-05-01     90.733533
2016-06-01    119.360231
2016-07-01    107.158353
2016-08-01     85.569942
2016-09-01     86.030505
2016-10-01     90.017048
2016-11-01    120.675929
2016-12-01    127.206672
2017-01-01    127.107377
2017-02-01    104.839699
2017-03-01    100.993663
2017-04-01     89.947354
2017-05-01     83.871739
2017-06-01     97.709238
2017-07-01     90.243697
2017-08-01     98.058954
2017-09-01     98.726182
2017-10-01     89.830472
2017-11-01     84.812229
2017-12-01     86.662940
Name: Korea_EPU, dtype: float64

In [18]:
pd.DataFrame(korea_epu).reset_index().to_csv('one_2015_to_2017.csv',index=False)

# 논문에 있던 다른 키워드 추가
- 직접개발 0.370857	< barker 0.412254

In [19]:
# 1) 경제(E): 거시경기·실물경제를 직접 지칭
e = [
    "경제", "경기"
]

# 2) 정책(P): 정부·의회·당국 및 모든 정책·제도·규제·조치
p = [
    # ─ 기본 정책 키워드 ─
    "정부", "청와대", "국무회의", "국회", "의회", "당국",
    "한국은행", "한은", "중앙은행",
    "기획재정부", "기재부", "금융위원회", "금융위",
    "정책", "재정", "입법", "법안", "법률", "예산",
    "세금", "규제", "규정", "적자", "부채", "채무",
    "연방준비제도", "연준", "Fed", "FRB",
    "구조개혁", "구조조정",

    # ─ 통화·금융 정책 ─
    "통화정책", "금융완화", "추가완화", "양적완화", "질적완화",
    "통화긴축", "유동성긴축", "마이너스 금리", "통화할인율", "재할인율",
    "통화 운용", "공개시장조작", "물가안정목표", "물가목표",
    "금융통화위원회", "금통위", "금융통화운영위원회", "금융통의",
    "기준금리", "정책금리", "금리정책", "통화당국", "환금리",

    # ─ 재정 정책 ─
    "재정정책", "정부 예산", "추가경정예산", "추경", "일반 회계", "특별 회계",
    "재정적자", "재정수지", "정부지출", "재정지출",
    "사회보장성 지출", "국민연금 보험료", "국민건강 보험료",
    "의료비 지출", "간병비 지출", "의료보수수가", "의료수가",
    "공무원급여", "공적개발원조", "ODA",
    "국방비", "군비",
    "국채 발행 잔여", "공공부문 부채", "재정부채", "국채", "정부부채", "지방채",
    "경기부양", "경기부양책",

    # ─ 무역·통상 정책 ─
    "통상문제", "무역문제", "비관세장벽", "수입제한",
    "포괄통상법", "종합무역법",
    "무역정책", "통상정책", "무역협상",
    "세계무역기구", "WTO",
    "관세 및 무역에 관한 일반협정", "GATT", "가트",
    "관세 인하", "무역자유화", "수입자유화", "시장접근",
    "무역협정", "통상협정",
    "환태평양경제동반자협정", "TPP",
    "경제동반자협정", "경제파트너협정", "경제 파트너십 협정", "EPA",
    "자유무역협정", "FTA",
    "무역분쟁", "관세", "우루과이라운드", "도하라운드", "덤핑",

    # ─ 환율 정책 ─
    "외환정책", "환율정책", "시장개입", "외환시장 개입", "외환당국 개입",
    "미세조정", "환율 안정책", "환율 안정 조치", "환율 변동성 완화",
    "환율 조작", "외환보유고", "외환보유액", "달러매도", "원화 매입 개입"
]

# 3) 불확실(U): 모호성·위험·걱정·우려를 표현
u = [
    "불확실", "불확실성", "리스크", "불투명",
    "불안", "우려", "걱정"
]


In [20]:
import re

cols = ['키워드', '특성추출(가중치순 상위 50개)', '본문', '제목']          # 검색 대상 열
joined = df[cols].fillna('').agg(' '.join, axis=1)                          # 네 열을 하나로 합침

cond1 = joined.str.contains('|'.join(map(re.escape, e)))                    # 경제·경기
cond2 = joined.str.contains('|'.join(map(re.escape, p)))                    # 불확실 관련
cond3 = joined.str.contains('|'.join(map(re.escape, u)))                    # 정책(정부 등)
                 # 정책(정부 등)

# 세 조건을 모두 만족하는 기사만 추리기
df_epu = df[cond1&cond2&cond3]

In [21]:
print(df_epu.shape, '데이터를 사용하게 되었습니다')
print('전체의', df_epu.shape[0]/df.shape[0] *100, '%입니다.')

(83932, 21) 데이터를 사용하게 되었습니다
전체의 1.9331972242593543 %입니다.


In [22]:
df_epu.columns

Index(['Unnamed: 0', '뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2',
       '통합 분류3', '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관',
       '키워드', '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부', 'file_date_range'],
      dtype='object')

In [23]:
# 신문사 월별로 집계
df_epu['일자'] = pd.to_datetime(df_epu['일자'], format='%Y%m%d')
df_epu['일자']= df_epu['일자'].dt.strftime('%Y-%m-01')
집계 = pd.DataFrame(df_epu.groupby(['일자','언론사']).size().reset_index())
집계.columns = ['일자','언론사','epu기사수']

C:\Users\samsung\AppData\Local\Temp\ipykernel_26620\353623932.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epu['일자'] = pd.to_datetime(df_epu['일자'], format='%Y%m%d')
C:\Users\samsung\AppData\Local\Temp\ipykernel_26620\353623932.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epu['일자']= df_epu['일자'].dt.strftime('%Y-%m-01')


In [24]:
# 각 신문의 상대 기사 수
전체집계 = pd.DataFrame(df.groupby(['일자','언론사']).size().reset_index())
전체집계['일자'] = pd.to_datetime(전체집계['일자'], format='%Y%m%d')
전체집계['일자'] = 전체집계['일자'].dt.strftime('%Y-%m-01')
전체집계.columns =  ['일자','언론사','전체기사수']
전체집계 = 전체집계.groupby(['일자','언론사'])['전체기사수'].sum().reset_index()
전체집계= 전체집계.merge(집계, on=['일자','언론사'])
전체집계['상대빈도수'] = 전체집계['epu기사수']/전체집계['전체기사수']

In [25]:
# 각 신문의 상대 EPU 수를 전체 표본 기간의 단위 표준 편차로 표준화
전체집계['표준화빈도'] = 전체집계['상대빈도수'] / 전체집계.groupby('언론사')['상대빈도수'].transform('std')

In [26]:
# 신문의 표준화 시리즈를 월별로 평균하여 전체 월간 한국 EPU지수
전체집계['일자'] = pd.to_datetime(전체집계['일자'], format='%Y-%m-01')

# ② 신문 간 단순 평균 → 월별 EPU
monthly_epu = (전체집계
               .groupby('일자')['표준화빈도']
               .mean())

In [27]:
# EPU 지수를 표본 기간 중 평균 100으로 정규화
korea_epu = (monthly_epu / monthly_epu.mean() * 100).rename('Korea_EPU')

In [28]:
korea_epu

일자
2015-01-01    102.144195
2015-02-01     82.260171
2015-03-01     89.591435
2015-04-01     77.580952
2015-05-01     80.388121
2015-06-01    137.655520
2015-07-01     92.712175
2015-08-01    109.846744
2015-09-01     95.976677
2015-10-01     83.842428
2015-11-01     86.868019
2015-12-01    117.203779
2016-01-01    144.655272
2016-02-01    131.544943
2016-03-01     89.708074
2016-04-01     99.722775
2016-05-01     90.725724
2016-06-01    121.354998
2016-07-01    107.153846
2016-08-01     85.028904
2016-09-01     84.416950
2016-10-01     89.381130
2016-11-01    121.364690
2016-12-01    128.417024
2017-01-01    127.737323
2017-02-01    104.237863
2017-03-01    100.099913
2017-04-01     89.860972
2017-05-01     83.659804
2017-06-01     98.145861
2017-07-01     89.507957
2017-08-01     98.863345
2017-09-01     99.234806
2017-10-01     88.432933
2017-11-01     84.649404
2017-12-01     86.025275
Name: Korea_EPU, dtype: float64

In [29]:
pd.DataFrame(korea_epu).reset_index().to_csv('two_2015_to_2017.csv',index=False)

# KDI 한국개발연구원 키워드만
- 직접개발 0.500216 > barker 0.412254 	

In [30]:
# 1) 경제(E): 거시경기·실물경제를 직접 지칭
e = [
    '경제','경제의','상업','무역'
]

# 2) 정책(P): 정부·의회·당국 및 모든 정책·제도·규제·조치
p = [
    '정부', '청와대', '국회', '당국', '한국은행', '중앙은행', '한은', '기재부', '기획재정부', '세계무역기구'
    , 'WTO', '제정', '재정법', '입법', '세금', '규제', '규정', '적자', '부족'
]

# 3) 불확실(U): 모호성·위험·걱정·우려를 표현
u = [
    '불확실성','불확'
]


In [31]:
import re

cols = ['키워드', '특성추출(가중치순 상위 50개)', '본문', '제목']          # 검색 대상 열
joined = df[cols].fillna('').agg(' '.join, axis=1)                          # 네 열을 하나로 합침

cond1 = joined.str.contains('|'.join(map(re.escape, e)))                    # 경제·경기
cond2 = joined.str.contains('|'.join(map(re.escape, p)))                    # 불확실 관련
cond3 = joined.str.contains('|'.join(map(re.escape, u)))                    # 정책(정부 등)
                 # 정책(정부 등)

# 세 조건을 모두 만족하는 기사만 추리기
df_epu = df[cond1&cond2&cond3]

In [32]:
print(df_epu.shape, '데이터를 사용하게 되었습니다')
print('전체의', df_epu.shape[0]/df.shape[0] *100, '%입니다.')

(3644, 21) 데이터를 사용하게 되었습니다
전체의 0.08393188158510563 %입니다.


In [33]:
df_epu.columns

Index(['Unnamed: 0', '뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2',
       '통합 분류3', '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관',
       '키워드', '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부', 'file_date_range'],
      dtype='object')

In [34]:
# 신문사 월별로 집계
df_epu['일자'] = pd.to_datetime(df_epu['일자'], format='%Y%m%d')
df_epu['일자']= df_epu['일자'].dt.strftime('%Y-%m-01')
집계 = pd.DataFrame(df_epu.groupby(['일자','언론사']).size().reset_index())
집계.columns = ['일자','언론사','epu기사수']

C:\Users\samsung\AppData\Local\Temp\ipykernel_26620\353623932.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epu['일자'] = pd.to_datetime(df_epu['일자'], format='%Y%m%d')
C:\Users\samsung\AppData\Local\Temp\ipykernel_26620\353623932.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epu['일자']= df_epu['일자'].dt.strftime('%Y-%m-01')


In [35]:
# 각 신문의 상대 기사 수
전체집계 = pd.DataFrame(df.groupby(['일자','언론사']).size().reset_index())
전체집계['일자'] = pd.to_datetime(전체집계['일자'], format='%Y%m%d')
전체집계['일자'] = 전체집계['일자'].dt.strftime('%Y-%m-01')
전체집계.columns =  ['일자','언론사','전체기사수']
전체집계 = 전체집계.groupby(['일자','언론사'])['전체기사수'].sum().reset_index()
전체집계= 전체집계.merge(집계, on=['일자','언론사'])
전체집계['상대빈도수'] = 전체집계['epu기사수']/전체집계['전체기사수']

In [36]:
# 각 신문의 상대 EPU 수를 전체 표본 기간의 단위 표준 편차로 표준화
전체집계['표준화빈도'] = 전체집계['상대빈도수'] / 전체집계.groupby('언론사')['상대빈도수'].transform('std')

In [37]:
# 신문의 표준화 시리즈를 월별로 평균하여 전체 월간 한국 EPU지수
전체집계['일자'] = pd.to_datetime(전체집계['일자'], format='%Y-%m-01')

# ② 신문 간 단순 평균 → 월별 EPU
monthly_epu = (전체집계
               .groupby('일자')['표준화빈도']
               .mean())

In [38]:
# EPU 지수를 표본 기간 중 평균 100으로 정규화
korea_epu = (monthly_epu / monthly_epu.mean() * 100).rename('Korea_EPU')

In [39]:
korea_epu

일자
2015-01-01     82.855053
2015-02-01     78.914957
2015-03-01     49.327322
2015-04-01     39.154369
2015-05-01     56.888790
2015-06-01     59.574665
2015-07-01     49.940267
2015-08-01     78.891061
2015-09-01    122.093731
2015-10-01     83.781760
2015-11-01     67.427367
2015-12-01    118.779825
2016-01-01    129.140112
2016-02-01     96.599794
2016-03-01     68.283376
2016-04-01     54.155040
2016-05-01     43.796704
2016-06-01    204.595290
2016-07-01    145.955004
2016-08-01     34.361479
2016-09-01     54.758661
2016-10-01     55.213776
2016-11-01    331.372935
2016-12-01    291.985495
2017-01-01    304.345459
2017-02-01    232.494672
2017-03-01    136.951070
2017-04-01     85.743823
2017-05-01     89.853219
2017-06-01     53.164348
2017-07-01     38.376761
2017-08-01     55.689720
2017-09-01     56.787748
2017-10-01     39.952627
2017-11-01     48.048301
2017-12-01     60.745420
Name: Korea_EPU, dtype: float64

In [40]:
pd.DataFrame(korea_epu).reset_index().to_csv('three_2015_to_2017.csv',index=False)